# <center>L2 Computational Physics</center>
---

## Week 5: Monte Carlo Methods

In this notebook, you will write a program that employs analytical, ODE, and Monte Carlo methods to simulate the progression of a collection of atoms through the radioactive decay channel $^{225}\textrm{Ra} \rightarrow~^{225}\textrm{Ac} \rightarrow~^{221}\textrm{Fr}$

In [1]:
import numpy
from matplotlib import pyplot as plt
from random import uniform
import random
from scipy.integrate import odeint

In [2]:
# Define all constants to be used in the simulation
t_half_ra = 20.8 # Half life of Ra-225 in days
t_half_ac = 10.0 # Half life of Ac-225 in days
N0 = 250  # Initial number of Ra-225 atoms
t1 = 100.  # End time for simulation in days
n_time = 50  # Number of timepoints to use

Firstly, analytically derive the number of $^{225}\textrm{Ra}$ atoms at each timepoint in the continuum limit. Implement this in the function `analytical(n0, t)`.

In [3]:
def analytical(n0, t):
    return n0*(0.5)**(t/t_half_ra)

Check your answer:

In [4]:
assert numpy.isclose(analytical(N0, t1), 8.926483655724745)

Implement a function that tells whether a decay has occured, based on the probability for the decay and a random number. Use the random number `r` from `random.random()` so that the checks can work in a reproducible way.

In [11]:
def has_decayed(prob):
    r = random.random()
    if prob > r:
        return True
    else:
        return False

In [12]:
random.seed(9867)
assert [ has_decayed(0.5) for i in range(10)] == [False, False, True, False, False, False, False, True, False, True]

We are going to keep track of the state of the atoms using a list that will contain either `'Ra'`, `'Ac'` or `Fr` for each atom. For example 
```python
state = ['Ra', 'Ra', 'Ac']
```
Define a function that takes such a state and implements the decay of an `initial` type of atoms into a new type `final` with probability `prob` and returns the new state

In [36]:
def evolve(state, initial, final, prob):
    length = len(state)
    for i in range(length):
        if state[i]==initial:
            if has_decayed(prob):
                state[i]=final
            else:
                length=length
        else:
            length=length
    return state

In [37]:
# check that we only decay the initial atoms
assert evolve(['Ra']*10, 'notRa', 'Ac', 0.5) == (['Ra']*10)
# check that if prob = 1 all decay
assert evolve(['Ra']*10, 'Ra', 'Ac', 1.0) == (['Ac'])*10
# check that if prob = 0 none decay
assert evolve(['Ra']*10, 'Ra', 'Ac', 0.0) == (['Ra'])*10
random.seed(3927)
assert evolve(['Ra']*10, 'Ra', 'Ac', 0.5) == ['Ra', 'Ac', 'Ac', 'Ac', 'Ra', 'Ra', 'Ac', 'Ra', 'Ac', 'Ac']

Define a function that evolves a system that starts with `N0` Radon atoms and evolved it in `n_timestep` from time $t=0$ to $t=t_{max}$. The function should return three arrays, one for each atom type. Each array should contain `n_timestep+1` elements including the initial amount. 

In order for the marking system to work make sure you update the `Ra` atoms first, the `Ac` next and the `Fr` atoms last. 

In [42]:
def evolve_system(tmax, n_timestep):
    state = ['Ra'] * N0
    timestep = tmax/n_timestep
    TauRa = t_half_ra*1.44269504
    TauAc = t_half_ac*1.44269504
    ProbRa = (1/TauRa)*numpy.exp(-timestep/TauRa)
    ProbAc = (1/TauAc)*numpy.exp(-timestep/TauAc)
    ra_count = numpy.empty(n_timestep + 1, dtype=int)
    ac_count = numpy.empty(n_timestep + 1, dtype=int)
    fr_count = numpy.empty(n_timestep + 1, dtype=int)
    ra_count[0] = N0
    ac_count[0] = 0
    fr_count[0] = 0
    for i in range (1,n_timestep+1):
        state1 = evolve(state,'Ra','Ac',ProbRa)
        state = evolve(state1,'Ac','Fr',ProbAc)
        ra_count[i] = list(state).count('Ra')
        ac_count[i] = list(state).count('Ac')
        fr_count[i] = list(state).count('Fr')
    
    return ra_count, ac_count, fr_count

In [45]:
r1, r2, r3 = evolve_system(10, 17)
assert len(r1) == 18
assert len(r2) == 18
assert len(r3) == 18

In [46]:
random.seed(9485)
r1, r2, r3 = evolve_system(10, 10)
assert (r1 == [250, 239, 229, 222, 216, 208, 203, 197, 190, 183, 179]).all()
assert (r2 == [ 0, 11, 21, 24, 28, 31, 35, 38, 44, 49, 53]).all()
assert (r3 == [ 0,  0,  0,  4,  6, 11, 12, 15, 16, 18, 18]).all()

AssertionError: 

## Plotting tasks

Create a plot with the number of Ra, Ac and Fr atoms, starting with `N0` Ra atoms and evolving the system for 100 hours using 1000 steps. The plot should have labels, a legend and a title.    


In [ ]:
nsteps = 1000
ra, ac, fr = evolve_system(100, nsteps)
ts = numpy.linspace(0,100,nsteps+1)
plt.plot(ts,ra, label='Ra')
plt.plot(ts,ac, label='Ac')
plt.plot(ts,fr, label='Fr')
plt.legend()
plt.xlabel('time [h]')
plt.ylabel('Number of atoms');

Run the simulation 100 times with 100 steps and use the results to calculate an average and the uncertainty on the number of Fr atoms as a function of time. Use and `errorbar` plot for it. You might be interested in the `numpy.average` and `numpy.std` functions. The plot should have axis labels and a title.  


In [ ]:
nsteps = 50
ts = numpy.linspace(0,100,nsteps+1)

values = numpy.empty( (100,nsteps+1))
for i in range(100):
    ra, ac, fr = evolve_system(100, nsteps)
    values[i] = fr

averages = numpy.average(values, axis=0)
uncertainties = numpy.std(values, axis=0)
plt.errorbar(ts,averages, yerr=uncertainties, color='b', alpha=0.4);

In [ ]:
nsteps = 150
ts = numpy.linspace(0,100,nsteps+1)
for i in range(100):
    ra, ac, fr = evolve_system(100, nsteps)
    plt.plot(ts,fr, color='b', alpha=0.04)